In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Pedestrian sensor data file
pedestrian_sensor_locations_path = "../Datasets/pedestrian-counting-system-sensor-locations.csv"

# Read the pedestrian sensor data file
pedestrian_sensor_locations = pd.read_csv(pedestrian_sensor_locations_path)

# Import the OpenWeatherMap API key
#from api_keys import city_of_melbourne_api_key

pedestrian_sensor_locations_df = pd.DataFrame(pedestrian_sensor_locations)
pedestrian_sensor_locations_df

In [ ]:
import hvplot.pandas
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts


sensors_map = pedestrian_sensor_locations_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    color = "Status",
    alpha = 0.5,
    hover_cols=["location_id", "Sensor_description"]
)
print(f"Number of sensors in the list: {len(pedestrian_sensor_locations_df)}")
sensors_map

In [ ]:
# Filter rows with "Outdoor" value in "location_type" column
outdoor_pedestrian_sensors_df = pedestrian_sensor_locations_df[pedestrian_sensor_locations_df["location_type"] == "Outdoor"]

# List of sensors in Melbourne CBD
location_ids = [1, 2, 3, 4, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 30, 36, 39, 40, 41, 45, 47, 48, 49, 51, 52, 53, 55, 56, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 75, 79, 84, 88, 89, 98, 99, 107, 108, 109, 113] 

cbd_pedestrian_sensors_df = outdoor_pedestrian_sensors_df.loc[outdoor_pedestrian_sensors_df["location_id"].isin(location_ids)]

cbd_pedestrian_sensors_df = cbd_pedestrian_sensors_df.reset_index()
print(f"Number of sensors in the list: {len(cbd_pedestrian_sensors_df)}")
cbd_pedestrian_sensors_df


In [ ]:
cbd_sensors_map = cbd_pedestrian_sensors_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    color = "Status",
    alpha = 0.5,
    hover_cols=["location_id", "Sensor_description"]
)

print(f"Number of sensors in the list: {len(cbd_pedestrian_sensors_df)}")
cbd_sensors_map

In [ ]:
# Define the list of location IDs from pedestrian_sensor_locations_df
location_ids = cbd_pedestrian_sensors_df['location_id'].tolist()

# Set the API base URL
url = 'https://data.melbourne.vic.gov.au/api/'

# Define an empty list to fetch the pedestrian count for each sensor
pedestrian_count = []

# Create counters
record_count = 1
set_count = 1

# Define the endpoint
endpoint = '/records/1.0/search/'

# Loop through all the location_ids in our list to pedestrian count
for location_id in location_ids:
    # Define the API parameters
    params = {
        'dataset': 'pedestrian-counting-system-past-hour-counts-per-minute',
        'q': '',
        'sort': 'sensingdatetime',
        'facet': 'locationid',
        'facet': 'sensingdatetime',
        'refine.locationid': location_id
    }
    # Send the API request
    response = requests.get(url + endpoint, params=params)

    # Check the status code of the response
    if response.status_code == 200:
        # Extract the pedestrian count from the response
        pedestrian_count.append(response.json()['nhits'])
        # Print the record count and location ID
        print(f"Record {record_count} of Set {set_count} | Location ID: {location_id} | Pedestrian Count: {response.json()['nhits']}")
    else:
        # Print an error message if the API request fails
        print(f"Error: API request failed for Location ID {location_id}")

    # Increment the record count
    record_count += 1

In [ ]:
# Define the list of location IDs from pedestrian_sensor_locations_df
location_ids = cbd_pedestrian_sensors_df['location_id'].tolist()

# Set the API base URL
url = 'https://data.melbourne.vic.gov.au/api/'

# Define an empty list to fetch the pedestrian count for each sensor
pedestrian_count = []

# Create counters
record_count = 1
set_count = 1

# Define the endpoint
endpoint = '/records/1.0/search/'

# Loop through all the location_ids in our list to pedestrian count
for location_id in location_ids:
    # Define the API parameters
    params = {
    'dataset': 'pedestrian-counting-system-past-hour-counts-per-minute',
    'q': '',
    'sort': 'sensingdatetime',
    'facet': 'locationid',
    'facet': 'sensingdatetime',
    'refine.locationid': location_id,
    'refine.sensingdatetime': '2022-04-01T00:00:00Z to 2022-04-01T23:59:59Z'
    }   
    # Send the API request
    response = requests.get(url + endpoint, params=params)

    # Check the status code of the response
    if response.status_code == 200:
        # Extract the pedestrian count from the response
        pedestrian_count.append(response.json()['nhits'])
        # Print the record count and location ID
        print(f"Record {record_count} of Set {set_count} | Location ID: {location_id} | Pedestrian Count: {response.json()['nhits']}")
    else:
        # Print an error message if the API request fails
        print(f"Error: API request failed for Location ID {location_id}")

    # Increment the record count
    record_count += 1
